# Kleven Best (2012) Replication

In [1]:
# Load libraries
using Parameters
using CSV
using DataFrames
using Plots
using Trapz
using NumericalIntegration
using SpecialFunctions

In [ ]:
# (Some imports / temporary)
# using Pkg
# Pkg.add("SpecialFunctions")

## Data setup

### Load and organize data.

The columns are:
* $z$: Income
* $H_y(z)$: CDF of income among the young
* $h_y(z)$: PDF of income among the young
* $H_o(z)$: CDF of income among the old
* $h_o(z)$: PDF of income among the old
* $mtr(z)$: Marginal tax rate at $z$
* $\omega$: Wage rate when old, as a function of ability and effort when young
* $j(\omega)$: PDF of $\omega$
* $J(\omega)$: CDF of $\omega$
* dtapars: Other parameters

The data parameters are:
* e: Static earnings elasticity w.r.t marginal tax rate
* d: Elasticity of old wages to young effort, $\delta = \frac{\partial w}{\partial z_y}\frac{z_y}{\omega}$
* $z_y$min: Minimum $z$ parameter for young Pareto distribution
* $\alpha_{zy}$: $\alpha$ parameter for young Pareto distribution
* $z_o$min: Minimum $z$ parameter for old Pareto distribution
* $\alpha_{zo}$: $\alpha$ parameter for old Pareto distribution
* $\gamma$: SWF inquality aversion
* $R$: Exogeneous revenue requirement

In [2]:
# Load data
statae050 = DataFrame(CSV.File("data/statae050.csv", header = false));

ndta = size(statae050)[1] # Data size

# Rename columns
rename!(statae050, [:z, :Hzy, :hzy, :Hzo, :hzo, :mtrz, :ω, :jω, :Jω, :dtapars]);


In [5]:
# Add a struct of parameters from the data
@with_kw struct dta_params
    e
    d ::Float64 = 0
    zymin
    αzy
    zomin
    αzo
    γ ::Float64 = 10
    R ::Int64 = 4000
end;

In [7]:
dtapars = dta_params(e = statae050.dtapars[1],
                     zymin = statae050.dtapars[2],
                     αzy = statae050.dtapars[3],
                     zomin = statae050.dtapars[4],
                     αzo = statae050.dtapars[5])

dta_params
  e: Float64 0.5
  d: Float64 0.0
  zymin: Float64 31043.303
  αzy: Float64 2.4402911
  zomin: Float64 36500.813
  αzo: Float64 2.2297311
  γ: Float64 10.0
  R: Int64 4000


### Define the remaining functions.

1.) $\omega(n, z_y/n)$: Function relating wage rate when old to ability and effort when young.

We assume that this wage rate is parametrized by:

$$\omega = \omega(n, z_y/n) = \omega_0(n) \cdot \left(\frac{z_y}{n}\right)^\delta $$

Where $w_0(n)$ is the baseline old wage for ability $n$ with no investment effects $(\delta = 0)$.

With an assumption about $\delta$ and data on $z_y$, we can back out $\omega_0(n)$ which is the wage that satisfies $\omega(.) = \omega$ for each individual.

In [ ]:
# ω(n, z_y/n)


In [ ]:
statae050.dtapars[1:10]

## Optimal tax calculation

### Procedure

In [ ]:
# Set parameters
# Set some parameters
@with_kw struct params
    γ ::Int64            # SWF inequality aversion parameter
    k ::Int64            # Parametrizes elasticity ε
    e ::Float64 = 1/k    # Elasticity ε = 1/k
    a ::Int64            # Pareto parameter for upper income distribution
    R ::Int64            # Exogenous per-person revenue requirement
end;